# Transformers with Keras: Translation
https://keras.io/examples/audio/transformer_asr/

In [1]:
from tensorflow.compat.v1 import keras
from tensorflow.compat.v1.keras import backend as K
import tensorflow.compat.v1 as tf
from tensorflow.compat.v1.keras import layers


import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:

from tensorflow.core.protobuf import rewriter_config_pb2
#tf.disable_eager_execution()

# set GPU config
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement=True
config.allow_soft_placement=True

In [3]:
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
# os.environ["CUDA_VISIBLE_DEVICES"]= "0" # first gpu
#os.environ["CUDA_VISIBLE_DEVICES"]= "" # second gpu
# os.environ["CUDA_VISIBLE_DEVICES"]= "2" # third gpu

config = tf.ConfigProto()
G = len(tf.config.experimental.list_physical_devices('GPU'))
print("Num GPUs Available: ", G)
print("tf version ", tf.__version__)
print("keras version ", tf.keras.__version__)

Num GPUs Available:  2
tf version  2.6.0
keras version  2.6.0


In [4]:
os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'


from tensorflow.keras.mixed_precision import experimental as mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

loss_scale = policy.loss_scale
print('Loss scale: %s' % loss_scale)


# default is 1e-7 which is too small for float16.  Without adjusting the epsilon, we will get NaN predictions because of divide by zero problems
float_type = 'float16'
print("keras backend float size before: ", K.floatx())
K.set_epsilon(1e-4) 
K.set_floatx(float_type)

# set the new configs
sess = tf.Session(config=config)
K.set_session(sess)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPUs will likely run quickly with dtype policy mixed_float16 as they all have compute capability of at least 7.0
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale
Compute dtype: float16
Variable dtype: float32
Loss scale: DynamicLossScale(current_loss_scale=32768.0, num_good_steps=0, initial_loss_scale=32768.0, increment_period=2000, multiplier=2.0)
keras backend float size before:  float32


In [5]:

G = len(tf.config.experimental.list_physical_devices('GPU'))
print("Num GPUs Available: ", G)
print("tf version ", tf.__version__)
print("keras version ", tf.keras.__version__)
print("keras backend float size: ", K.floatx())

Num GPUs Available:  2
tf version  2.6.0
keras version  2.6.0
keras backend float size:  float16


# Data loading

In [6]:

x_train = np.load('../data/x_train.npy')
y_train = np.load('../data/y_train.npy')
x_val = np.load('../data/x_val.npy')
y_val = np.load('../data/y_val.npy')

print(x_train.dtype)
x_train = x_train.astype('int32')
print(x_train.dtype)


y_train = y_train.astype('int32')
x_val = x_val.astype('int32')
y_val = y_val.astype('int32')





# OVERFITTING TEST

x_train = x_train[0:2, :]
y_train = y_train[0:2, :]
x_val = x_train
y_val = y_train


print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")


print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)


float32
int32
2 Training sequences
2 Validation sequences
(2, 50, 60)
(2, 50, 60)
(2, 50, 60)
(2, 50, 60)


# Hyperparams

In [7]:
## Data shape
NUM_FEATS = 60
SEQ_LEN = 50
vocab_size = 300 #max index of discrete data
maxlen = SEQ_LEN#*NUM_FEATS # Setting this to reflect sequence length


## Model dimensions
EMBED_DIM = 5  # Embedding size for each token
NUM_HEADS = 5  # Number of attention heads
ff_dim = 512  # Hidden layer size in feed forward network inside transformer
dense_dim = 20

## Training params

patience = 10
BS = 2*G
num_epochs = 50000

In [8]:
def create_tf_dataset(data, bs=4):
    ds = data.map(lambda x, y: {"source": x, "target": y})
    ds = ds.batch(bs)
    return ds


train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))


DS = create_tf_dataset(train_dataset, bs=BS)
VAL_DS = create_tf_dataset(test_dataset, bs=BS)

<DatasetV1Adapter shapes: ((50, 60), (50, 60)), types: (tf.int32, tf.int32)>
4


## Learning rate schedule

In [9]:
class CustomSchedule(keras.optimizers.schedules.LearningRateSchedule):
    def __init__(
        self,
        init_lr=0.00001,
        lr_after_warmup=0.001,
        final_lr=0.00001,
        warmup_epochs=15,
        decay_epochs=85,
        steps_per_epoch=203,
    ):
        super().__init__()
        self.init_lr = init_lr
        self.lr_after_warmup = lr_after_warmup
        self.final_lr = final_lr
        self.warmup_epochs = warmup_epochs
        self.decay_epochs = decay_epochs
        self.steps_per_epoch = steps_per_epoch

    def calculate_lr(self, epoch):
        """ linear warm up - linear decay """
        warmup_lr = (
            self.init_lr
            + ((self.lr_after_warmup - self.init_lr) / (self.warmup_epochs - 1)) * epoch
        )
        decay_lr = tf.math.maximum(
            self.final_lr,
            self.lr_after_warmup
            - (epoch - self.warmup_epochs)
            * (self.lr_after_warmup - self.final_lr)
            / (self.decay_epochs),
        )
        return tf.math.minimum(warmup_lr, decay_lr)

    def __call__(self, step):
        epoch = step // self.steps_per_epoch
        return self.calculate_lr(epoch)


# Embedding

In [10]:
class TokenEmbedding(layers.Layer):
    def __init__(self, num_vocab=1000, maxlen=100, num_hid=64):
        super().__init__()
        self.emb = tf.keras.layers.Embedding(num_vocab, num_hid)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=num_hid)

    def call(self, x):
        print("TOKEN EMBEDDING WAS CALLED")
        maxlen = tf.shape(x)[-1]
        x = self.emb(x)
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        return x + positions


class SpeechFeatureEmbedding(layers.Layer):
    def __init__(self, num_hid=64, maxlen=100):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv1D(
            num_hid, 11, strides=2, padding="same", activation="relu"
        )
        self.conv2 = tf.keras.layers.Conv1D(
            num_hid, 11, strides=2, padding="same", activation="relu"
        )
        self.conv3 = tf.keras.layers.Conv1D(
            num_hid, 11, strides=2, padding="same", activation="relu"
        )
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=num_hid)

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        return self.conv3(x)

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size+1, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen+1, output_dim=embed_dim)

    def call(self, x):
        #x_shape = x.shape
        #print("hello I am embedding")
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        #print("embedding went fine!")
        return x + positions


# Encoder

In [11]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, rate=0.3):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [
                layers.Dense(feed_forward_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        #print("Hello I am ENCODING")
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)



# Decoder

In [12]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, dropout_rate=0.3):
        super().__init__()
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
        self.self_att = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, #attention_axes=(0,)
        )
        #print(f"hello before mha {embed_dim}")
        self.enc_att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim
                                                )
        #print("hello after")
        self.self_dropout = layers.Dropout(0.5)
        self.enc_dropout = layers.Dropout(0.1)
        self.ffn_dropout = layers.Dropout(0.1)
        self.ffn = keras.Sequential(
            [
                layers.Dense(feed_forward_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        
    

    def causal_attention_mask(self, batch_size, n_dest, n_src, dtype):
        """Masks the upper half of the dot product matrix in self attention.

        This prevents flow of information from future tokens to current token.
        1's in the lower triangle, counting from the lower right corner.
        """
        i = tf.range(n_dest)[:, None]
        j = tf.range(n_src)
        m = i >= j - n_src + n_dest
        mask = tf.cast(m, dtype)
        mask = tf.reshape(mask, [1, n_dest, n_src])
        mult = tf.concat(
            [tf.expand_dims(batch_size
                            , -1), tf.constant([1, 1], dtype=tf.int32
                                             )], 0
        )
        
        mask = tf.tile(mask, mult)
        return mask
        
        
    
    
    def test_mask_3D(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length,feats = input_shape[0], input_shape[1], input_shape[2]
        bs = inputs.shape[0]
        sl = inputs.shape[1]
        f = inputs.shape[2]
        mask = np.zeros((sl,f, sl, f),dtype="int32")
        for ind in range(sl):
            mask[ind,:,0:ind+1,:] += 1       

        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1, 1], dtype=tf.int32)],
            axis=0,
        )

        ret = tf.tile(mask, mult)
        return ret
        

            
    def call(self, enc_out, target):
        input_shape = target.shape
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        
        test_mask = self.test_mask_3D(target)
        target_att = self.self_att(target, target, attention_mask=test_mask)
        
        target_norm = self.layernorm1(target + self.self_dropout(target_att))
        enc_out = self.enc_att(target_norm, enc_out)
        enc_out_norm = self.layernorm2(self.enc_dropout(enc_out) + target_norm)
        ffn_out = self.ffn(enc_out_norm)
        ffn_out_norm = self.layernorm3(enc_out_norm + self.ffn_dropout(ffn_out))
        return ffn_out_norm


# Transformer 

In [20]:

class Transformer(keras.Model):
    
    def __init__(
        self,
        num_hid=64,
        num_head=2,
        num_feed_forward=128,
        source_maxlen=100,
        target_maxlen=100,
        num_layers_enc=6,
        num_layers_dec=1,
        num_classes=101,
    ):
        super().__init__()
        self.loss_metric = keras.metrics.Mean(name="loss")
        self.num_layers_enc = num_layers_enc
        self.num_layers_dec = num_layers_dec
        self.target_maxlen = target_maxlen
        self.num_classes = num_classes
        
        self.enc_input = TokenEmbedding(num_vocab=num_classes, maxlen=target_maxlen, num_hid=num_hid)
     
        self.dec_input = TokenEmbedding(num_vocab=num_classes, maxlen=target_maxlen, num_hid=num_hid)

        self.encoder = keras.Sequential(
            [self.enc_input]
            + [
                TransformerEncoder(num_hid, num_head, num_feed_forward)
                for _ in range(num_layers_enc)
            ]
        )

        for i in range(num_layers_dec):
            setattr(
                self,
                f"dec_layer_{i}",
                TransformerDecoder(num_hid, num_head, num_feed_forward),
            )


        self.classifier = layers.Dense(num_classes)

    def decode(self, enc_out, target):
        y = self.dec_input(target)
        for i in range(self.num_layers_dec):
            y = getattr(self, f"dec_layer_{i}")(enc_out, y)
        return y

    def call(self, inputs):

        
        source = inputs[0] #input[0] use source/target if eager is disabled
        target = inputs[1]
        
        x = self.encoder(source)
        y = self.decode(x, target)
        return self.classifier(y)

    @property
    def metrics(self):
        return [self.loss_metric]

    def train_step(self, batch):
        """Processes one batch inside model.fit()."""
        source = batch["source"]
        target = batch["target"]
        #print(f"TRAIN STEP source: {source}")
        #print(f"TRAIN STEP target: {target}")
        
        dec_input = target[:, :-1]
        dec_target = target[:, 1:]
        #print(f"TRAIN STEP dec_input: {dec_input}")
        #print(f"TRAIN STEP dec_target: {dec_target}")
        
        with tf.GradientTape() as tape:
            preds = self([source, dec_input])
            one_hot = tf.one_hot(dec_target, depth=self.num_classes)
            mask = tf.math.logical_not(tf.math.equal(dec_target, 0))
            loss = self.compiled_loss(one_hot, preds, sample_weight=mask)
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        self.loss_metric.update_state(loss)
        
        return {"loss": self.loss_metric.result()}

    def test_step(self, batch):
        print("*****TEST STEP WAS CALLED")
        source = batch["source"]
        target = batch["target"]
        dec_input = target[:, :-1]
        dec_target = target[:, 1:]
        preds = self([source, dec_input])
        one_hot = tf.one_hot(dec_target, depth=self.num_classes)
        mask = tf.math.logical_not(tf.math.equal(dec_target, 0))
        loss = self.compiled_loss(one_hot, preds, sample_weight=mask)
        self.loss_metric.update_state(loss)
        return {"loss": self.loss_metric.result()}

    def generate(self, source, target_start_token_idx):
        """Performs inference over one batch of inputs using greedy decoding."""
        print("****GENERATE STEP WAS CALLED")
        bs = tf.shape(source)[0]
        enc = self.encoder(source)
        dec_input = tf.ones((bs, 1), dtype=tf.int32) * target_start_token_idx
        dec_logits = []
        for i in range(self.target_maxlen - 1):
            dec_out = self.decode(enc, dec_input)
            logits = self.classifier(dec_out)
            logits = tf.argmax(logits, axis=-1, output_type=tf.int32)
            last_logit = tf.expand_dims(logits[:, -1], axis=-1)
            dec_logits.append(last_logit)
            dec_input = tf.concat([dec_input, last_logit], axis=-1)
        return dec_input


# Training

In [ ]:

loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1,)


learning_rate = CustomSchedule(
       init_lr=0.00001,
        lr_after_warmup=0.0001,
        final_lr=0.00001,
        warmup_epochs=30,
        decay_epochs=55,
        steps_per_epoch= x_train.shape[0]/BS,
    )



optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

mirrored_strategy = tf.distribute.MirroredStrategy()

with mirrored_strategy.scope():
    model = Transformer(
            num_hid=EMBED_DIM,
            num_head=NUM_HEADS,
            num_feed_forward=ff_dim,
            target_maxlen=maxlen,
            source_maxlen=maxlen,
            num_layers_enc=6,
            num_layers_dec=6,
            num_classes=vocab_size,
        )
    
    model.compile(optimizer=optimizer, loss=loss_fn, metrics=["accuracy"])


early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=patience, verbose=1)
checkpoint_filepath = '/models/checkpoint/overfit_test'
checkpoint = keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=False,
        monitor='val_loss',
        mode='min',
        save_best_only=True, verbose=1)


history = model.fit(DS, validation_data=VAL_DS, callbacks=[early_stopping, checkpoint], epochs=num_epochs, shuffle=True)
model.summary()


In [ ]:
# BUG: accuracy not available even when compiling with accuracy metric. 

print(history.history.keys())


# Testing masking functions

In [ ]:
# original masking function
def causal_attention_mask(inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        print(mask)
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        print(mult)
        ret = tf.tile(mask, mult)
        #print(ret)
        return ret

test = np.ones((4,10,3)) # bs, seq_len, embed_dim
mask = causal_attention_mask(test)
#print(mask)


In [ ]:
# working 3D mask variant
def test_mask(inputs):
    # test mask shape:
    input_shape = tf.shape(inputs)
    batch_size, sequence_length,feats = input_shape[0], input_shape[1], input_shape[2]
    bs = inputs.shape[0]
    sl = inputs.shape[1]
    f = inputs.shape[2]
    mask = np.zeros((sl,f, sl, f),dtype="int32")
    for ind in range(sl):
        mask[ind,:,0:ind+1,:] += 1       
            
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1, 1], dtype=tf.int32)],
        axis=0,
    )

    ret = tf.tile(mask, mult)
    return ret

test = np.ones((1,5, 3, 2)) # bs, seq_len, feats, embed_dim
mask = test_mask(test)
print(mask)

# Generate

In [21]:
reconstructed_model = keras.models.load_model("transformer3D_overfit")

In [46]:
# OVERFIT TEST 3D


source = x_train[0:1]
target = y_train[0:1] 
print(source.shape)
target_start_token_idx = 0 # start of target sequence

preds = model.generate(source, target_start_token_idx)
preds = preds.numpy()
print(preds.shape)
print("preds: ", preds)

(1, 50, 60)
****GENERATE STEP WAS CALLED
TOKEN EMBEDDING WAS CALLED
Trying to decode - generate
TOKEN EMBEDDING WAS CALLED


InvalidArgumentError: Expected input 0 to have rank 4 but got: 3 [Op:Einsum]

In [ ]:
# get test data
x_test = np.load('../data/x_test.npy')
y_test = np.load('../data/y_test.npy')

x_test = x_test.astype('int32')
y_test = y_test.astype('int32')


source = x_test[0:1]
target = y_test[0:1]

target_start_token_idx = 0 # start of target sequence

preds = model.generate(source, target_start_token_idx)
preds = preds.numpy()
print(preds.shape)
print(preds)



In [ ]:
print(preds[0].shape)

In [ ]:
import csv

def write_ex_to_tsv(ex, fn):
    #num_frames = ex.shape[0]
    
    # squished
    num_frames = 50 
    
    freq = 30
    marker_names = ['MARKER_NAMES','Head','neck','lsho','lelb','lwri','lhan','rsho','relb','rwri','rhan','t10','root','lhip','lknee','lank','lfoot','rhip','rknee','rank','rfoot']#['MARKER_NAMES','Head','neck','rsho','relb','rwri','rhan','lsho','lelb','lwri','lhan','back','root','rhip','lhip','rknee','lknee','rank','lank', 'rfoot', 'lfoot']
    
    with open('../animations/tsv/'+fn+'.tsv', 'wt') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        tsv_writer.writerow(['NO_OF_FRAMES', num_frames])
        tsv_writer.writerow(['NO_OF_CAMERAS', 0])
        tsv_writer.writerow(['NO_OF_MARKERS', 20])
        tsv_writer.writerow(['FREQUENCY', freq])
        tsv_writer.writerow(['NO_OF_ANALOG', 0])
        tsv_writer.writerow(['ANALOG_FREQUENCY', 0])
        tsv_writer.writerow(['DESCRIPTION--', ''])
        tsv_writer.writerow(['TIME_STAMP--', ''])
        tsv_writer.writerow(['DATA_INCLUDED', '3D'])
        tsv_writer.writerow(marker_names)
        
        for frame in range(num_frames):
            tsv_writer.writerow(ex[frame,:])
            

In [ ]:
write_ex_to_tsv(preds[0], 'test-overfit3D')